In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
#!pip install pyjson
#!pip install pandas
#!pip install geopandas
#!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
#!pip install -U pandas-profiling

In [3]:
%cd /gdrive/My Drive/Data Quality

/gdrive/My Drive/Data Quality


In [4]:
import random
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
import tensorflow as tf

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer

from sklearn import preprocessing

tfk = tf.keras
tfkl = tf.keras.layers

In [5]:
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [6]:
def check_datatypes(df):
    for col in df.columns:
        if (df[col].dtype == "bool"):
            df[col] = df[col].astype('string')
            df[col] = df[col].astype('object')
    return  df

def injection(df_pandas, seed, name, name_class):

    np.random.seed(seed)

    #%%

    df_list = []

    perc = [0.50, 0.40, 0.30, 0.20, 0.10]
    for p in perc:
        df_dirt = df_pandas.copy()
        comp = [p,1-p]
        df_dirt = check_datatypes(df_dirt)

        for col in df_dirt.columns:

            if col!=name_class:

                rand = np.random.choice([True, False], size=df_dirt.shape[0], p=comp)

                df_dirt.loc[rand == True,col]=np.nan

        df_list.append(df_dirt)
        print("saved {}-completeness{}%".format(name, round((1-p)*100)))
    return df_list


In [7]:
adult = pd.read_csv("adult.csv")
adult

,age,workclass,fnlwgt,education,education-num,maritial-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,42,Self-emp-not-inc,99185,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
1,55,Private,163083,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,>50K
2,49,Private,340755,Masters,14,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States,>50K
3,31,Private,208881,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
4,19,Private,271446,Some-college,10,Never-married,Other-service,Own-child,White,Female,0,0,25,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3012,37,Local-gov,160910,Bachelors,13,Married-civ-spouse,Adm-clerical,Husband,White,Male,7298,0,40,United-States,>50K
3013,30,Private,488720,9th,5,Married-civ-spouse,Handlers-cleaners,Other-relative,White,Male,0,0,40,Mexico,<=50K
3014,37,Private,138940,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
3015,34,Private,225548,Bachelors,13,Married-civ-spouse,Protective-serv,Husband,Black,Male,0,0,30,United-States,<=50K


In [8]:
#profile = ProfileReport(adult, title="Report")
#profile.to_file("df_report.html")

In [9]:
df_list= injection(adult, seed, "adult", "income")
df_Nan = pd.DataFrame(df_list[4], columns=adult.columns)
#d = {'<=50K': 0, '>50K': 1}
#df_Nan["income"] = df_Nan["income"].map(d)
print(df_Nan.isnull().sum().sum())
df_Nan

saved adult-completeness50%
saved adult-completeness60%
saved adult-completeness70%
saved adult-completeness80%
saved adult-completeness90%
4335


,age,workclass,fnlwgt,education,education-num,maritial-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,42.0,Self-emp-not-inc,99185.0,NaN,10.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
1,55.0,Private,163083.0,Masters,14.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
2,49.0,Private,340755.0,Masters,14.0,NaN,Sales,Husband,White,Male,0.0,0.0,40.0,NaN,>50K
3,31.0,NaN,208881.0,Some-college,10.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0.0,0.0,40.0,United-States,<=50K
4,19.0,Private,271446.0,Some-college,10.0,Never-married,Other-service,Own-child,White,Female,NaN,0.0,NaN,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3012,37.0,Local-gov,160910.0,Bachelors,13.0,Married-civ-spouse,Adm-clerical,Husband,White,Male,7298.0,0.0,40.0,United-States,>50K
3013,30.0,NaN,488720.0,9th,5.0,Married-civ-spouse,Handlers-cleaners,Other-relative,White,Male,0.0,0.0,40.0,Mexico,<=50K
3014,NaN,Private,138940.0,11th,7.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0.0,0.0,NaN,United-States,<=50K
3015,34.0,Private,225548.0,Bachelors,13.0,Married-civ-spouse,Protective-serv,Husband,Black,Male,NaN,NaN,NaN,United-States,<=50K


In [10]:
df_Nan.dtypes

age                float64
workclass           object
fnlwgt             float64
education           object
education-num      float64
maritial-status     object
occupation          object
relationship        object
race                object
sex                 object
capital-gain       float64
capital-loss       float64
hours-per-week     float64
native-country      object
income              object
dtype: object

In [11]:
#profile = ProfileReport(df_Nan, title="Report")
#profile.to_file("df_report_Nan.html")

# Advanced Imputation technique


In [12]:
df = df_Nan.copy()
le = preprocessing.LabelEncoder()
df1 = df[["workclass", "education", "maritial-status", "occupation", "relationship", "race", "sex", "native-country", "income"]]

df1 = df1.apply(lambda series: pd.Series(
    le.fit_transform(series[series.notnull()]),
    index=series[series.notnull()].index
))
df1

,workclass,education,maritial-status,occupation,relationship,race,sex,native-country,income
0,4.0,NaN,2.0,2.0,0.0,4.0,1.0,36.0,1
1,2.0,12.0,2.0,8.0,0.0,4.0,1.0,36.0,1
2,2.0,12.0,NaN,10.0,0.0,4.0,1.0,NaN,1
3,NaN,15.0,2.0,1.0,0.0,4.0,1.0,36.0,0
4,2.0,15.0,4.0,6.0,3.0,4.0,0.0,36.0,0
...,...,...,...,...,...,...,...,...,...
3012,1.0,9.0,2.0,0.0,0.0,4.0,1.0,36.0,1
3013,NaN,6.0,2.0,4.0,2.0,4.0,1.0,23.0,0
3014,2.0,1.0,2.0,1.0,0.0,4.0,1.0,36.0,0
3015,2.0,9.0,2.0,9.0,0.0,2.0,1.0,36.0,0


In [13]:
df2 = df[["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]]
df_advanced = pd.concat([df1, df2], axis=1)
df_advanced

,workclass,education,maritial-status,occupation,relationship,race,sex,native-country,income,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,4.0,NaN,2.0,2.0,0.0,4.0,1.0,36.0,1,42.0,99185.0,10.0,0.0,0.0,40.0
1,2.0,12.0,2.0,8.0,0.0,4.0,1.0,36.0,1,55.0,163083.0,14.0,0.0,0.0,40.0
2,2.0,12.0,NaN,10.0,0.0,4.0,1.0,NaN,1,49.0,340755.0,14.0,0.0,0.0,40.0
3,NaN,15.0,2.0,1.0,0.0,4.0,1.0,36.0,0,31.0,208881.0,10.0,0.0,0.0,40.0
4,2.0,15.0,4.0,6.0,3.0,4.0,0.0,36.0,0,19.0,271446.0,10.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3012,1.0,9.0,2.0,0.0,0.0,4.0,1.0,36.0,1,37.0,160910.0,13.0,7298.0,0.0,40.0
3013,NaN,6.0,2.0,4.0,2.0,4.0,1.0,23.0,0,30.0,488720.0,5.0,0.0,0.0,40.0
3014,2.0,1.0,2.0,1.0,0.0,4.0,1.0,36.0,0,NaN,138940.0,7.0,0.0,0.0,NaN
3015,2.0,9.0,2.0,9.0,0.0,2.0,1.0,36.0,0,34.0,225548.0,13.0,NaN,NaN,NaN


In [14]:
multivariate_impute_pipe = ColumnTransformer([
    ("impute_num", IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=1),
                                    max_iter=50), df_advanced.columns)])
df_mice = multivariate_impute_pipe.fit_transform(df_advanced)
df_mice = pd.DataFrame(df_mice, columns=df_advanced.columns)
df_mice.to_csv(index=False)
df_mice

,workclass,education,maritial-status,occupation,relationship,race,sex,native-country,income,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,4.0,8.0,2.0,2.0,0.0,4.0,1.0,36.0,1.0,42.0,99185.0,10.0,0.0,0.0,40.0
1,2.0,12.0,2.0,8.0,0.0,4.0,1.0,36.0,1.0,55.0,163083.0,14.0,0.0,0.0,40.0
2,2.0,12.0,2.0,10.0,0.0,4.0,1.0,23.0,1.0,49.0,340755.0,14.0,0.0,0.0,40.0
3,2.0,15.0,2.0,1.0,0.0,4.0,1.0,36.0,0.0,31.0,208881.0,10.0,0.0,0.0,40.0
4,2.0,15.0,4.0,6.0,3.0,4.0,0.0,36.0,0.0,19.0,271446.0,10.0,0.0,0.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3012,1.0,9.0,2.0,0.0,0.0,4.0,1.0,36.0,1.0,37.0,160910.0,13.0,7298.0,0.0,40.0
3013,2.0,6.0,2.0,4.0,2.0,4.0,1.0,23.0,0.0,30.0,488720.0,5.0,0.0,0.0,40.0
3014,2.0,1.0,2.0,1.0,0.0,4.0,1.0,36.0,0.0,22.0,138940.0,7.0,0.0,0.0,50.0
3015,2.0,9.0,2.0,9.0,0.0,2.0,1.0,36.0,0.0,34.0,225548.0,13.0,0.0,0.0,40.0


In [15]:
df = adult.copy()
le = preprocessing.LabelEncoder()
df1 = df[["workclass", "education", "maritial-status", "occupation", "relationship", "race", "sex", "native-country", "income"]]

df1 = df1.apply(lambda series: pd.Series(
    le.fit_transform(series[series.notnull()]),
    index=series[series.notnull()].index
))

df2 = df[["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]]
df_adult= pd.concat([df1, df2], axis=1)
df_adult

,workclass,education,maritial-status,occupation,relationship,race,sex,native-country,income,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,4,15,2,2,0,4,1,36,1,42,99185,10,0,0,40
1,2,12,2,8,0,4,1,36,1,55,163083,14,0,0,40
2,2,12,2,10,0,4,1,36,1,49,340755,14,0,0,40
3,2,15,2,1,0,4,1,36,0,31,208881,10,0,0,40
4,2,15,4,6,3,4,0,36,0,19,271446,10,0,0,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3012,1,9,2,0,0,4,1,36,1,37,160910,13,7298,0,40
3013,2,6,2,4,2,4,1,23,0,30,488720,5,0,0,40
3014,2,1,2,1,0,4,1,36,0,37,138940,7,0,0,40
3015,2,9,2,9,0,2,1,36,0,34,225548,13,0,0,30


In [16]:
total = 0
right = 0

for col in df_adult.columns:
  for row in range(0, df_adult.shape[0]):
    if col != "income":
      if pd.isna(df_advanced[col][row]):
        total = total + 1
        if df_adult[col][row] == df_mice[col][row]:
          right = right + 1

total, right
accuracy_standard = right/total
print(total)
print(right)
print(accuracy_standard)

4335
1917
0.4422145328719723


In [17]:
target = "income"
features = df_mice.columns[df_mice.columns != target]

x = df_mice[features]
y = df_mice[target]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=seed, shuffle=True, stratify=y)

# Logistic Regression

In [18]:
logistic = LogisticRegression(random_state=seed, solver="liblinear")
logistic.fit(x_train,y_train)
yp = logistic.predict(x_test)
logistic.classes_

array([0., 1.])

In [19]:
accuracy = accuracy_score(y_test, yp)
precision = precision_score(y_test, yp)
recall = recall_score(y_test, yp)
f1 = f1_score(y_test, yp)
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

cm = confusion_matrix(y_test, yp)
plt.figure(figsize=(10,8))
sns.heatmap(cm, cmap='Blues', xticklabels=["0", "1"], yticklabels=["0", "1"])
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

Accuracy: 0.7583
Precision: 0.5556
Recall: 0.1974
F1: 0.2913


# KNN

In [20]:
knn = KNeighborsClassifier(n_neighbors=5, algorithm='kd_tree')
knn.fit(x_train,y_train)
y_predicted = knn.predict(x_test)
knn.classes_

array([0., 1.])

In [21]:
accuracy = accuracy_score(y_test, y_predicted)
precision = precision_score(y_test, y_predicted)
recall = recall_score(y_test, y_predicted)
f1 = f1_score(y_test, y_predicted)
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

cm = confusion_matrix(y_test, y_predicted)
plt.figure(figsize=(10,8))
sns.heatmap(cm, cmap='Blues', xticklabels=["0", "1"], yticklabels=["0", "1"])
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

Accuracy: 0.7285
Precision: 0.4032
Recall: 0.1645
F1: 0.2336


# AdaBoost Classifier

In [22]:
ada_boost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), n_estimators=50)
ada_boost.fit(x_train,y_train)
y_predicted = ada_boost.predict(x_test)
ada_boost.classes_

array([0., 1.])

In [23]:
accuracy = accuracy_score(y_test, y_predicted)
precision = precision_score(y_test, y_predicted)
recall = recall_score(y_test, y_predicted)
f1 = f1_score(y_test, y_predicted)
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

cm = confusion_matrix(y_test, y_predicted)
plt.figure(figsize=(10,8))
sns.heatmap(cm, cmap='Blues', xticklabels=["0", "1"], yticklabels=["0", "1"])
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

Accuracy: 0.7914
Precision: 0.5878
Recall: 0.5724
F1: 0.58
